In [1]:
import os
os.environ["CEA_USE_SITE_PACKAGES"] = '1'
from CEA_Wrap import Fuel, Oxidizer, RocketProblem

from utils import bar_to_psi

Propellants

In [2]:
ethanol = Fuel("C3H8O,2propanol", temp=298)  # Liquid IPA at room temperature
n2o = Oxidizer("N2O", temp=298)        # Nitrous oxide at room temperature
# TODO: not sure if there's a way to pull the Stoichiometric out of these two, 
# would be neat to plug in to graph by mixture instead of phi

Rocket Problem

In [3]:
# See page 25 of manual for parameter description (.venv/lib/python3.13/site-packages/CEA_Wrap/assets/CEA_Users_Manual_and_Program_Description.pdf) 
problem = RocketProblem(
    pressure=bar_to_psi(20),    # Pressure taken in PSI
    materials=[ethanol, n2o],   # Propellants
    phi=1.2,                    # (actual F/O) / (stoichiometric F/O)
    sup=1.6,                    # Expansion ratio (sup (supersonic, nozzle): Ae/At, sub (subsonic, chamber): Ac/At)
)

Run the analysis

In [4]:
results = problem.run()

Key results

In [5]:
print(f"Chamber Temperature (K):     {results.c_t:.1f}")
print(f"Chamber Pressure (bar):      {results.c_p:.1f}")
print(f"Exit Temperature (K):        {results.t:.1f}")
print(f"Exit Pressure (bar):         {results.p:.1f}")
print(f"Specific Impulse (s):        {results.isp:.1f}")
print(f"Characteristic Velocity (m/s): {results.cstar:.1f}")
print(f"Thrust Coefficient:          {results.cf:.3f}")

Chamber Temperature (K):     3195.8
Chamber Pressure (bar):      20.0
Exit Temperature (K):        2658.4
Exit Pressure (bar):         3.7
Specific Impulse (s):        180.9
Characteristic Velocity (m/s): 1594.0
Thrust Coefficient:          1.113


Exhaust composition (top 5 species)

In [6]:
print("\n=== Major Exhaust Products (mol fraction) ===")
products = [(species, fraction) for species, fraction in results.prod_e.items()]
products.sort(key=lambda x: x[1], reverse=True)

for species, fraction in products[:5]:
    print(f"{species:10}: {fraction:.4f}")


=== Major Exhaust Products (mol fraction) ===
N2        : 0.5093
H2O       : 0.2382
CO2       : 0.1164
CO        : 0.0879
H2        : 0.0272
